<a href="https://colab.research.google.com/github/abelowska/mlNeuro/blob/main/MLN_spatial_filters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spatial filters

See the MNE wonderful tutorials and documantation to better understand spatial filters: [https://mne.tools/stable/auto_tutorials/machine-learning/50_decoding.html#spatial-filters](https://mne.tools/stable/auto_tutorials/machine-learning/50_decoding.html#spatial-filters)


In [ ]:
!pip install mne

Imports

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mne
from mne.datasets import eegbci
from mne.datasets import sample
from mne.decoding import UnsupervisedSpatialFilter, CSP, Vectorizer


from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.decomposition import PCA, FastICA

## Read data

In [ ]:
data_dir = Path('./data')
epochs_subjects = []

for idx in np.arange(1,11):
  fname = data_dir / f'subj_{idx}-epo.fif'
  print(fname)
  epochs = mne.read_epochs(fname)
  epochs_subjects.append(epochs)

## Visualize data

In [ ]:
all_epochs = mne.concatenate_epochs(epochs_subjects)

1. Joint plot per condition

In [ ]:
fig = all_epochs['left'].average().plot_joint(times=[-1, 0.5, 0.8, 1.5, 2,3,4])
fig = all_epochs['right'].average().plot_joint(times=[-1, 0.5, 0.8, 1.5, 2,3,4])

2. Single-channel plots

In [ ]:
picks = ['C3', 'Cz', 'C4']

evokeds = dict(
    left=list(all_epochs["left"].iter_evoked()),
    right=list(all_epochs["right"].iter_evoked()),
)

for idx, pick in enumerate(picks):
  plt.figure(idx)
  fig = mne.viz.plot_compare_evokeds(evokeds, picks=pick)
  plt.show()

## Spatial filters

### PCA

1. Fit PCA to all data

In [ ]:
# define PCA
n_components = 2
pca = UnsupervisedSpatialFilter(PCA(n_components), average=False)

# get data to train PCA
X = all_epochs.get_data(copy=False)

# fit PCA
pca.fit(X)

# extract sklearn PCA object
pca_estimator = pca.estimator

2. Explore PCA

Note, that PCA components has shape of *(n_components, n_channels)*, as each channel is a sample for PCA.

In [ ]:
# explained variance of X
print(pca_estimator.explained_variance_ratio_)

# shape of PCA components
print(pca_estimator.components_.shape)

You can use mne tool for vizualization to see the PCA weights that were estimated for each channel, i.e., patterns. They show how each channel contribute to the given PCA component.

In [ ]:
for i in np.arange(0, n_components):
  # create canvas
  plt.figure(i)

  # get data of i-th component
  spatial_data = pca_estimator.components_[i]
  mne.viz.plot_topomap(
      spatial_data,
      pos=all_epochs.info,
      show=False
  )

  plt.show()

3. Transform data using fitted PCA

In [ ]:
X_filtered = pca.transform(X)

print(f'X shape befor PCA: {X.shape}\nX shape after PCA: {X_filtered.shape}')

We can compare the waves of single-channel X with the pattern of the first (main) PCA component

In [ ]:
# plot evoke for selected channels from X
c3_index = all_epochs.info.ch_names.index('C3')
fz_index = all_epochs.info.ch_names.index('Fz')

X_mean = np.mean(X, axis=0)
plt.plot(X_mean[c3_index])
plt.plot(X_mean[fz_index])


# plot evoke for the first component from X_filtered
pca_component_index = 0
X_filtered_mean = np.mean(X_filtered, axis=0)
plt.plot(X_filtered_mean[pca_component_index])

The easiest way to check if PCA captured the difference between the conditions, and thus is suitable for our analysis, is to wrap data transformed with PCA to `Epochs` object that store info on events.

In [ ]:
# original data
print(all_epochs)

# create info for transformed data
events = all_epochs.events

info = mne.create_info(
    n_components,
    epochs.info['sfreq'],
    ch_types='eeg'
)

all_epochs_pca = mne.EpochsArray(
    X_filtered,
    info,
    events,
    tmin=all_epochs.tmin
)
print(all_epochs_pca)

Now we can use all MNE methods for all_epochs_pca visualization.

In [ ]:
# original data
picks = ['C3']

evokeds = dict(
    left=list(all_epochs["left"].iter_evoked()),
    right=list(all_epochs["right"].iter_evoked()),
)

fig = mne.viz.plot_compare_evokeds(evokeds, picks=picks)

In [ ]:
# PCA filtered data
picks = ['0']

evokeds_pca = dict(
    left=list(all_epochs_pca["1"].iter_evoked()),
    right=list(all_epochs_pca["2"].iter_evoked()),
)

fig = mne.viz.plot_compare_evokeds(evokeds_pca, picks=picks)

PCA does not seem to be the best approach we can adopt ;)

### CSP

1. Fit CSP to all data

Note that because CSP is a supervised method, we have to provide the *y* set to CSP.

In [ ]:
# define PCA
n_components = 2
csp = CSP(n_components)

# get data to train CSP
X = all_epochs.get_data(copy=False)
y = all_epochs.events[:, -1] - 1

# fit CSP
csp.fit(X, y)

2. Explore CSP

Using build-in functions we can easily plot patterns (CSP weights; also called mixing matrix) and filters (CSP weights/patterns multiplied by the original data).

In [ ]:
fig = csp.plot_patterns(all_epochs.info)

In [ ]:
fig = csp.plot_filters(all_epochs.info)

Note, that the forst CSP pattern is quite similar to our forst PCA component/pattern that captures most of the variance in X!

3. Transform the original data

In [ ]:
X_filtered_csp = csp.transform(X)

print(f'X shape befor PCA: {X.shape}\nX shape after PCA: {X_filtered_csp.shape}')

CSP does not preserve the time dimension; it averages the signal along the time domain. As a result of CSP, we obtain as many features as the number of CSP components we declare.

## Classification

In [ ]:
def estimate_model(
    X_train,
    X_test,
    y_train,
    y_test,
    model=SVC()
):
  # fit
  model.fit(X_train, y_train)

  # predict test and train data
  y_test_predicted = model.predict(X_test)
  y_train_predicted = model.predict(X_train)

  print(f'Classification report for testing data:\n{classification_report(y_test, y_test_predicted)}')
  print(f'Classification report for training data:\n{classification_report(y_train, y_train_predicted)}')

  return model

Create train and test sets

In [ ]:
def train_test_split_epochs(epochs_list, split=0.7):
  train_n = int(len(epochs_list)*split)
  test_n = len(epochs_list) - train_n
  train_epochs = mne.concatenate_epochs(epochs_list[:train_n])
  test_epochs = mne.concatenate_epochs(epochs_list[-test_n:])

  X_train = train_epochs.get_data(copy=True)
  X_test = test_epochs.get_data(copy=True)
  y_train = train_epochs.events[:, -1] - 1
  y_test = test_epochs.events[:, -1] - 1

  return X_train, y_train, X_test, y_test

### 1. PCA-based features


1. Just pipe all transformed data to the model. Note that `UnsupervisedSpatialFilter` object is a transformer, so we can use it within `pipelines`. This helps prevent knowledge leakage from the training set to the test set when performing PCA.

In [ ]:
# define X and y sets
X_train, y_train, X_test, y_test = train_test_split_epochs(
    epochs_subjects,
    split=0.7
)

print(X_train.shape)

In [ ]:
n_components = 2

model = make_pipeline(
    UnsupervisedSpatialFilter(PCA(n_components), average=False),
    Vectorizer(),  # vectorize across time and channels to n_samples, n_features
    StandardScaler(),
    SVC(),
)

_ = estimate_model(
    X_train,
    X_test,
    y_train,
    y_test,
    model=model
)

##### Exercise
2. Extract the mean amplitude within a selected time-window from the first PCA component. Note that in the previous example, we performed **all data transformations within pipelines**. Now, you have the option to write your own transformer to extract the mean amplitude from each 'channel' (e.g., https://medium.com/@pgshanding/creating-custom-transformers-in-python-and-scikit-learn-10767487017e) or perform PCA outside of the pipelines.

In [ ]:
# define X and y sets
X_train, y_train, X_test, y_test = train_test_split_epochs(
    epochs_subjects,
    split=0.7
)

In [ ]:
# your code here

model = make_pipeline()

_ = estimate_model(
    X_train,
    X_test,
    y_train,
    y_test,
    model=model
)

### CSP-based features

#### Exercise

Try to use CSP to extract *n* features from our data. Did CSP significantly improve the classification results? To avoid choosing n manually, you can use  [`GridSearch`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to find the optimal *n* value.

In [ ]:
# define X and y sets
X_train, y_train, X_test, y_test = train_test_split_epochs(
    epochs_subjects,
    split=0.7
)

In [ ]:
# your code here

# model = TODO

_ = estimate_model(
    X_train,
    X_test,
    y_train,
    y_test,
    model=model
)